# Import

In [1]:
import os 
import re
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Dataset

In [6]:
file_path = os.getenv('HOME') + '/aiffel/transformer_chatbot/data/ChatbotData .csv'

In [7]:
chatbot_df = pd.read_csv(file_path)
chatbot_df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


# 단어 전처리

In [8]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    
    return sentence

In [9]:
def load_conversations(questions, answers):
    inputs, outputs = [], []
    
    for question, answer in zip(questions, answers):
        inputs.append(preprocess_sentence(question))
        outputs.append(preprocess_sentence(answer))
        
    return inputs, outputs

In [14]:
questions, answers = load_conversations(chatbot_df['Q'], chatbot_df['A'])

# Subword TextEncoder

In [15]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [16]:
Start_Token, End_Token = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

In [17]:
print('Start_Token의 번호: ', [tokenizer.vocab_size])
print('End_Token의 번호: ', [tokenizer.vocab_size + 1])

Start_Token의 번호:  [8173]
End_Token의 번호:  [8174]


In [38]:
VOCAB_SIZE = tokenizer.vocab_size + 2

In [19]:
Max_Length = 40

In [24]:
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    
    for(sentence1, sentence2) in zip(inputs, outputs):
        sentence1 = Start_Token + tokenizer.encode(sentence1) + End_Token
        sentence2 = Start_Token + tokenizer.encode(sentence2) + End_Token
        
        if len(sentence1) <= Max_Length and len(sentence2) <= Max_Length:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
            
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen = Max_Length, padding = 'post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen = Max_Length, padding = 'post')
    
    return tokenized_inputs, tokenized_outputs

In [25]:
questions, answers = tokenize_and_filter(questions, answers)

# Transformer

In [26]:
class PositionalEncoding(tf.keras.layers.Layer):
    
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)
        
    def get_angles(self, position, i, d_model):
        angles = 1/tf.pow(10000, (2 * (i//2)) / tf.cast(d_model, tf.float32))
        return position * angles
    
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(position = tf.range(position, dtype = tf.float32)[:, tf.newaxis], i = tf.range(d_model, dtype = tf.float32)[tf.newaxis, :], d_model = d_model)
        
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
    
        pos_encoding = tf.stack([sines, cosines], axis = 0)
        pos_encoding = tf.transpose(pos_encoding, [1, 2, 0])
        pos_encoding = tf.reshape(pos_encoding, [position, d_model])
    
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)
    
    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [27]:
def scaled_dot_product_attention(query, key, value, mask):
    matmul_qk = tf.matmul(query, key, transpose_b = True)
    
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)
    
    if mask is not None:
        logits += (mask * -1e9)
        
    attention_weights = tf.nn.softmax(logits, axis = -1)
    
    output = tf.matmul(attention_weights, value)
    return output

In [28]:
class MultiHeadAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads, name = 'multi_head_attention'):
        super(MultiHeadAttention, self).__init__(name = name)
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        
        self.query_dense = tf.keras.layers.Dense(units = d_model)
        self.key_dense = tf.keras.layers.Dense(units = d_model)
        self.value_dense = tf.keras.layers.Dense(units = d_model)
        
        self.dense = tf.keras.layers.Dense(units = d_model)
        
    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape = (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm = [0, 2, 1, 3])
    
    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]
        
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)
        
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)
        
        scaled_attention = tf.transpose(scaled_attention, perm = [0, 2, 1, 3])
        
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        
        outputs = self.dense(concat_attention)
        
        return outputs
        

In [29]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [30]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 -tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [31]:
def encoder_layer(units, d_model, num_heads, dropout, name = 'encoder_layer'):
    inputs = tf.keras.Input(shape = (None, d_model), name = 'inputs')
    
    padding_mask = tf.keras.Input(shape = (1, 1, None), name = 'padding_mask')
    
    attention = MultiHeadAttention(d_model, num_heads, name = 'attention')({'query': inputs, 'key': inputs, 'value': inputs, 'mask': padding_mask})
    
    attention = tf.keras.layers.Dropout(rate = dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon = 1e-6)(inputs + attention)
    
    outputs = tf.keras.layers.Dense(units = units, activation = 'relu')(attention)
    outputs = tf.keras.layers.Dense(units = d_model)(outputs)
    
    outputs = tf.keras.layers.Dropout(rate = dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon = 1e-6)(attention + outputs)
    
    return tf.keras.Model(inputs = [inputs, padding_mask], outputs = outputs, name = name)

In [44]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [45]:
def decoder_layer(units, d_model, num_heads, dropout, name = 'decoder_layer'):
    inputs =tf.keras.Input(shape = (None, d_model), name = 'inputs')
    enc_outputs = tf.keras.Input(shape = (None, d_model), name = 'encoder_outputs')
    look_ahead_mask = tf.keras.Input(shape = (1, None, None), name = 'look_ahead_mask')
    padding_mask = tf.keras.Input(shape = (1, 1, None), name = 'padding_mask')
    
    attention1 = MultiHeadAttention(d_model, num_heads, name = 'attention1')(inputs = {'query': inputs, 'key': inputs, 'value': inputs, 'mask': look_ahead_mask})
    
    attention1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)(attention1 + inputs)
    
    attention2 = MultiHeadAttention(d_model, num_heads, name = 'attention2')(inputs = {'query': attention1, 'key': enc_outputs, 'value': enc_outputs, 'mask': padding_mask})
    
    attention2 = tf.keras.layers.Dropout(rate = dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)(attention2 + attention1)
                                    
    outputs = tf.keras.layers.Dropout(rate = dropout)(attention2)
    outputs = tf.keras.layers.LayerNormalization(epsilon = 1e-6)(outputs + attention2)
    
    return tf.keras.Model(inputs = [inputs, enc_outputs, look_ahead_mask, padding_mask], outputs = outputs, name = name)

In [46]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name='look_ahead_mask')

    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

# Train

In [47]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices(({'inputs': questions, 'dec_inputs': answers[:, :-1]}, {'outputs': answers[:, 1:]}))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [50]:
def transformer(vocab_size, num_layers, units, d_model, num_heads, dropout, name = 'transformer'):
    inputs = tf.keras.Input(shape = (None,), name = 'inputs')
    dec_inputs = tf.keras.Input(shape = (None,), name = 'dec_inputs')
    
    enc_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape = (1, 1, None), name = 'enc_padding_mask')(inputs)
    
    look_ahead_mask = tf.keras.layers.Lambda(create_look_ahead_mask, output_shape = (1, None, None), name = 'look_ahead_mask')(dec_inputs)
    
    dec_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape = (1, 1, None), name = 'dec_padding_mask')(inputs)
    
    enc_outputs = encoder(vocab_size = vocab_size, num_layers = num_layers, units = units, d_model = d_model, num_heads = num_heads, dropout = dropout)(inputs = [inputs, enc_padding_mask])
    
    dec_outputs = decoder(vocab_size = vocab_size, num_layers = num_layers, units = units, d_model = d_model, num_heads = num_heads, dropout = dropout)(inputs = [dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])
    
    outputs = tf.keras.layers.Dense(units = vocab_size, name = 'outputs')(dec_outputs)
    
    return tf.keras.Model(inputs = [inputs, dec_inputs], outputs = outputs, name = name)

In [51]:
tf.keras.backend.clear_session()

NUM_LAYERS = 6
D_MODEL = 512
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1

model = transformer(vocab_size = VOCAB_SIZE, num_layers = NUM_LAYERS, units = UNITS, d_model = D_MODEL, num_heads = NUM_HEADS, dropout = DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 512)    13653504    inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [65]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape= (-1, Max_Length - 1))
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')(y_true, y_pred)
    
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)
    
    return tf.reduce_mean(loss)

In [66]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [69]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1= 0.9, beta_2= 0.98, epsilon= 1e-9)


def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, Max_Length - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


model.compile(optimizer= optimizer, loss= loss_function, metrics= [accuracy])

In [70]:
EPOCHS = 50
model.fit(dataset, epochs = EPOCHS, verbose = 1)

Epoch 1/50
185/185 [==============================] - 62s 203ms/step - loss: 1.3411 - accuracy: 0.0227
Epoch 2/50
185/185 [==============================] - 39s 210ms/step - loss: 1.0848 - accuracy: 0.0472
Epoch 3/50
185/185 [==============================] - 38s 207ms/step - loss: 0.9864 - accuracy: 0.0507
Epoch 4/50
185/185 [==============================] - 39s 209ms/step - loss: 0.9390 - accuracy: 0.0529
Epoch 5/50
185/185 [==============================] - 39s 208ms/step - loss: 0.8941 - accuracy: 0.0560
Epoch 6/50
185/185 [==============================] - 39s 208ms/step - loss: 0.8356 - accuracy: 0.0600
Epoch 7/50
185/185 [==============================] - 39s 208ms/step - loss: 0.7688 - accuracy: 0.0655
Epoch 8/50
185/185 [==============================] - 39s 208ms/step - loss: 0.6961 - accuracy: 0.0722
Epoch 9/50
185/185 [==============================] - 39s 208ms/step - loss: 0.6213 - accuracy: 0.0802
Epoch 10/50
185/185 [==============================] - 39s 208ms/step - l

# 번역

In [71]:
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

    sentence = tf.expand_dims(
        Start_Token + tokenizer.encode(sentence) + End_Token, axis=0)

    output_sequence = tf.expand_dims(Start_Token, 0)

    for i in range(Max_Length):
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if tf.equal(predicted_id, End_Token[0]):
            break

        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)

In [72]:

def sentence_generation(sentence):
    prediction = decoder_inference(sentence)

    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size])

    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence

In [73]:
sentence_generation("롤 티어가 어디야?")

입력 : 롤 티어가 어디야?
출력 : 세상에는 예상하지 못한 일들이 자주 일어나는 것 같아요 .


'세상에는 예상하지 못한 일들이 자주 일어나는 것 같아요 .'